In [10]:
import requests
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import time
from pprint import pprint
import citipy
import json
from citipy import citipy

In [44]:
url="https://openweathermap.org/current"
req=requests.get(url)


#json_data['coord']
#weather_desc=["weather"][0]["description"]
print(req.text)

<!DOCTYPE html>
<html lang='en'>
    <head>
        <!-- smart banner apple -->
        <meta name="apple-itunes-app" content="app-id=1535923697">
        <script data-ad-client="ca-pub-6837309406068967" async src="https://pagead2.googlesyndication.com/pagead/js/adsbygoogle.js"></script>
        <meta http-equiv="Content-Type" content="text/html; charset=utf-8">    
    <meta  http-equiv=Expires content="Tue, Sep 20 2018 15:27:22 GMT">
    <meta http-equiv="Last-Modified" content="Tue, Sep 20 2018 15:27:22 GMT">
    <!--<meta http-equiv="X-UA-Compatible" content="IE=edge">-->
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <meta name="keywords" content="weather, world, Openstreetmap, forecast, Openweather, current" >
    <meta name="description" content="Access current weather data for any location on Earth. We collect and process weather data from different sources such as global and local weather models, satellites, radars and vast network of weather st

In [11]:
api_key="aeb3cf034d1ddda25b0e5ee5f3be4847" #or fd337120c2b9b99842d3a34fa258bbe1

In [12]:
print(api_key)

aeb3cf034d1ddda25b0e5ee5f3be4847


In [13]:
base_url="http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key
headers = {"Longitude": "lon", "Latitude": "lat", "Country": "country", "Wind Speed": "wind", "Highest Temp": "temp_max", "Pressure": "pressure", "Humidity": "humidity"}
city="london"
print(base_url + f'&q={city}')
#create a json output of the data
req = requests.get(base_url + f'&q={city}', headers=headers)
json_data = req.json()
print(json_data)

http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=aeb3cf034d1ddda25b0e5ee5f3be4847&q=london
{'coord': {'lon': -0.1257, 'lat': 51.5085}, 'weather': [{'id': 804, 'main': 'Clouds', 'description': 'overcast clouds', 'icon': '04n'}], 'base': 'stations', 'main': {'temp': 47.37, 'feels_like': 46.71, 'temp_min': 45.45, 'temp_max': 49.75, 'pressure': 997, 'humidity': 92}, 'visibility': 10000, 'wind': {'speed': 3, 'deg': 266, 'gust': 11.99}, 'clouds': {'all': 100}, 'dt': 1634786661, 'sys': {'type': 2, 'id': 2019646, 'country': 'GB', 'sunrise': 1634798083, 'sunset': 1634835312}, 'timezone': 3600, 'id': 2643743, 'name': 'London', 'cod': 200}


In [30]:
lat_lngs = []
cities = []

# Create a set of random lat and lng combinations
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)

# Identify nearest city for each lat, lng combination
for lat_lng in lat_lngs:
    city = citipy.nearest_city(lat_lng[0], lat_lng[1]).city_name
    
    # If the city is unique, then add it to a our cities list
    if city not in cities:
        cities.append(city)

# Print the city count to confirm sufficient count
len(cities)

641

In [35]:
#performing the calls
#tell it which variables to collect
#city_weather_list=["lon", "lat", "country"]
raw_weather = []

base_url="http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + api_key

try:
    for city in cities:
        req = requests.get(base_url + f'&q={city}')
        json_data = req.json()
        raw_weather.append(json_data)
        print("Processing "+ city)

except requests.exceptions.RequestException as e:
        print("Processing "+ city + "failed.")
   

#convert json_data to a dataframe that includes "lat" "long" "country" "wind" "temp_max" "pressure" "humidity"

#print(base_url + f'&q={city}')
#create a json output of the data
#coordinates, country, max temperature, pressure, humidity and wind speed



Processing east london
Processing chateaubelair
Processing longyearbyen
Processing culleredo
Processing port alfred
Processing bluff
Processing mataura
Processing hobart
Processing ushuaia
Processing taolanaro
Processing grand river south east
Processing georgetown
Processing rikitea
Processing cherskiy
Processing punta arenas
Processing mys shmidta
Processing qaanaaq
Processing lavrentiya
Processing vaitape
Processing linqiong
Processing chuy
Processing hami
Processing chunhuhub
Processing cape town
Processing jamestown
Processing antakya
Processing ancud
Processing hithadhoo
Processing praia da vitoria
Processing busselton
Processing yulara
Processing saint anthony
Processing upernavik
Processing mayo
Processing chokurdakh
Processing urazovka
Processing aguas formosas
Processing butaritari
Processing waimes
Processing atuona
Processing xam nua
Processing kirgiz-miyaki
Processing lompoc
Processing hasaki
Processing pilar
Processing artyk
Processing kapaa
Processing ribeira grande
Proc

Processing axim
Processing crixas
Processing marsa matruh
Processing sao raimundo nonato
Processing bouca
Processing santa marta
Processing bambanglipuro
Processing armacao dos buzios
Processing gravdal
Processing dalnerechensk
Processing sao filipe
Processing gladstone
Processing dauriya
Processing sawtell
Processing mumbwa
Processing santander
Processing mudigere
Processing canora
Processing geraldton
Processing yaan
Processing nurota
Processing sarh
Processing naze
Processing clyde river
Processing tenenkou
Processing poronaysk
Processing kalmunai
Processing taguatinga
Processing constitucion
Processing ivdel
Processing westpunt
Processing akureyri
Processing longido
Processing kamenka
Processing kargapolye
Processing nelson bay
Processing poddorye
Processing bilibino
Processing asau
Processing manama
Processing nancha
Processing wanning
Processing inyonga
Processing ambon
Processing ibra
Processing sangar
Processing valle de allende
Processing leshan
Processing kieta
Processing vic

In [27]:
cities_weather = pd.DataFrame(raw_weather)
cities_weather_tailored = cities_weather[["name","country", "lon", "lat", "wind", "temp_max", "pressure", "humidity"]]
print(cities_weather_tailored.head())


NameError: name 'raw_weather' is not defined

Latitude vs. Temperature Plot

In [ ]:
#build scatterplots
x="latitude"
y="temp_max"
sns.set_style("whitegrid")
sns.set_style("ticks")
sns.scatterplot(x="lat", y="temp_max", data=cities_weather)
plt.show()

Latitude vs. Humidity Plot

In [ ]:
x="latitude"
y="temp_max"
sns.set_style("whitegrid")
sns.set_style("ticks")
sns.scatterplot(x="lat", y="humidity", data=cities_weather)
plt.show()

Latitude vs. Cloudiness Plot

In [ ]:
x="latitude"
y="temp_max"
sns.set_style("whitegrid")
sns.set_style("ticks")
sns.scatterplot(x="lat", y="temp_max", data=cities_weather)
plt.show()

Latitude vs. Wind Speed Plot

In [ ]:
x="latitude"
y="temp_max"
sns.set_style("whitegrid")
sns.set_style("ticks")
sns.scatterplot(x="lat", y="wind", data=cities_weather)
plt.show()

In [ ]:
#Sources
#https://towardsdatascience.com/how-to-pull-data-from-an-api-using-python-requests-edcc8d6441b1